In [3]:
## Download data from 
## https://archive.ics.uci.edu/ml/datasets/Mice+Protein+Expression
## change the header of the last column 'class' to 'GBT'
## save the file in csv format
import pandas as pd

In [2]:
## Generate input (X) and output (y)
## Perform binary classification using Genotype as class label (79th column, index=78)
import numpy as np

In [3]:
## Preprocessing: missing values and normalization
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [4]:
## Use LabelEncoder for numerical encoding of class labels
from sklearn import preprocessing  

In [5]:
## Naive Bayes classifier on the transformed data
## performance evaluation: 5-fold cross validation 
## metric: auc
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB